# 자연어 처리 NLP (Natural Language Processing)

자연어 처리는 인간의 언어 현상을 컴퓨터와 같은 기계를 이용하여 모사할 수 있도록 연구하고 이를 구현하는 인공지능의 주요 분야 중 하나다.<br>
컴퓨터 공학과 언어학과 같은 다른 학문과의 융합적인 요소도 필요하며, 컴퓨터와 인간 사이의 연결고리를 만들어주는 것이 자연어 처리 분야라고 할 수 있다.

## 자연어 처리의 주 응용분야

* Sentiment Analysis - 감성분석, 인터넷 이용자의 글에서 감정을 분류하는 작업
* Quention & Answering - QA, 지문을 읽고 주어진 질문에 대해 대답하는 분야
* 애플의 Siri와 같이 사용자의 의도를 파악하고 대화하거나 도움을 주는 작업
* Summarization 요약, Machine Translation 기계번역과 같은 작업
* Language Modeling - 언어모델링, 첫 몇 마디만 입력하면 뒤에 나올 단어 예측해서 선택만 하면 되는 스마트보드

1950년부터 자연어를 처리하려는 시도가 있었고 그 뒤로 계속 문제점을 가지고 있다가 <br>
딥러닝이 발전하면서 2014년에 seq2seq(Sequence-to-Sequence)구조가 제시되면서 기계 번역은 신경망을 기반으로 하는 것이 대세가 됨

딥러닝이 처음에 이미지 처리 분야에서 강한 면모를 보였다면 자연어 처리는 최근에 가장 진척이 많이 나간 연구 분야이기도 함<br>
최근 모든 자연어 처리 문제에서 최고의 경지인 SOTA(State od the art)를 기록하고 있는 모델은 BERT(Bidirectional Encoder Representations from Transformers) <br> 

BERT는 "Attention is All You Need" 논문에 등장한 Trnasformer 모델의 Encoder 부분을 기반으로 대량의 코퍼스를 통해 비지도학습으로 범용적인 언어 모델을 학습한 뒤에 풀려고 하는 문제(Task)에 전이 학습(Transfer Learning)을 하는 방법임

## 딥러닝 자연어 처리의 일반적인 프로세스

딥러닝에서 자연어 처리를 수행하기 위해서 텍스트 형식의 데이터를 머신러닝 알고리즘이 이해할 수 있는 숫자<br>
즉, 벡터 형식의 데이터로 만들어 줘야 한다.

1. tokenization - 문장의 토큰화
2. vocabulary - 중복되지 않은 토근으로 vocabulary 생성 후, 숫자와 1대 1 대응
3. numericalization - 단어장(vocabulary)를 기반으로 토큰을 숫자로 전환 
4. padding - 길이가 다양한 문장을 동일한 길이로 만들어 주는 padding 과정

### 문장이란 tokenization

자연어 처리에서 제일 우선적으로 해야할 작업은 
* 문장(sentence)를 기계가 처리할 수 있는 데이터로 만드는 것

문장(sentence)는 길이가 다른 일련의 토큰(token)의 연속이다.
* 즉, '순서'라는 특징을 가진 시퀀스 데이터(sequence data)라고 할 수 있다.
* 여기서 토큰(token)은 `단어`가 될 수도 있고, `형태소`가 될 수도 있고 `하나의 문자열`이 될 수도 있다.
    * 형태소 : 의미를 가지는 요소로서는 더 이상 분석할 수 없는 가장 작은 말의 단위
    
Example: 문장(sentence) - "오늘 날씨 좋다"<br>

**tokenization**<br>
* token - 단어(띄어쓰기 기준) : '오늘' + '날씨' + '좋다'
* token - 형태소 : '오늘 ' + '날씨' + '좋' + '다'
* token - 문자열 : '오' + '늘' + ' ' + '날' + '씨' + ' ' + '좋' + '다'

**중요한 포인트**<br>
영어의 경우 띄어쓰기로 단어 위주의 토큰화를 할 수 있다.(nltk, SpaCy)(띄어쓰기 기준 split)
한글의 경우 띄어쓰기로 토큰화할 경우 단어 수가 많아지기 때문에 형태소 단위의 토큰화를 기본으로 한다.(KoNLPy)

In [1]:
from konlpy.tag import Okt

In [2]:
kor_sentence = "오늘 날씨 좋다"

In [3]:
tokenizer = Okt()

In [4]:
# morphs 함수로 형태소 분석
tokenizer.morphs(kor_sentence)

['오늘', '날씨', '좋다']

In [7]:
# pos 함수로 품사 태깅
tokenized_sent = tokenizer.pos(kor_sentence)
tokenized_sent

[('오늘', 'Noun'), ('날씨', 'Noun'), ('좋다', 'Adjective')]

In [10]:
# 영어로 된 품사 정보를 한글로 전환
[(k, tokenizer.tagset.get(v)) for k, v in tokenized_sent]

[('오늘', '명사'), ('날씨', '명사'), ('좋다', '형용사')]

### 단어장 생성 vocabulary

단어장(vocabulary)를 생성하는 과정은 중복되지 않는 토큰에 고유한 인덱스(index)를 부여하는 과정이다.
Python에서는 보통 dictionary 자료 구조로 단어장(vocabulary)을 생성한다.
네이버 감성 영화 코퍼스(naver sentiment movie corpus) 데이터를 사용해서 어떻게 진행되는지 알아본다.
이 데이터는 6만개의 댓글과 긍정 혹은 부정을 나타내는 라벨로 구성되어 있으며 그 사이는 tab으로 분리되어 있다.

In [11]:
from konlpy.tag import Okt

In [12]:
tokenizer = Okt()

In [16]:
with open("./data/jk_train.txt", encoding='UTF8') as file:
    raw_data = file.read().splitlines()[1:]
    data = [line.split("\t")[1:] for line in raw_data]
    data = [(tokenizer.morphs(sent), int(label)) for (sent, label) in data]